# Wavefront propagation simulation tutorial

L.Samoylova <liubov.samoylova@xfel.eu>, A.Buzmakov <buzmakov@gmail.com>

Tutorial course on Wavefront Propagation Simulations, 28/11/2013, European XFEL, Hamburg.

Wave optics software is based on SRW core library <https://github.com/ochubar/SRW>, available through WPG interactive framework <https://github.com/samoylv/WPG>

## Propagation Gaussian through horizontal offset mirror (HOM)

### Import modules

In [ ]:
%matplotlib inline

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

#Importing necessary modules:
import os
import sys
sys.path.insert(0,os.path.join('..', '..'))

import time
import copy
import numpy as np
import pylab as plt


#import SRW core functions
from wpg.srwlib import srwl,SRWLOptD,SRWLOptA,SRWLOptC,SRWLOptT,SRWLOptL

#import SRW helpers functions
from wpg.useful_code.srwutils import AuxTransmAddSurfHeightProfileScaled

#import some helpers functions
from wpg.useful_code.wfrutils import calculate_fwhm_x, plot_wfront, calculate_fwhm_y, print_beamline, get_mesh

#Import base wavefront class
from wpg import Wavefront

#Gaussian beam generator
from wpg.generators import build_gauss_wavefront_xy

plt.ion()

### Define auxiliary functions

In [ ]:
#Plotting
def plot_1d(profile, title_fig, title_x, title_y):
    plt.plot(profile[0], profile[1])
    plt.xlabel(title_x)
    plt.ylabel(title_y)
    plt.title(title_fig)
    plt.grid(True)


def plot_2d(amap, xmin, xmax, ymin, ymax, title_fig, title_x, title_y):
    plt.imshow(amap, extent=(ymin, ymax, xmin, xmax))
    plt.colorbar()
    plt.xlabel(title_x)
    plt.ylabel(title_y)
    plt.title(title_fig)


In [ ]:
#calculate source size from photon energy and FWHM angular divergence
def calculate_source_fwhm(ekev, theta_fwhm):
    wl = 12.39e-10/ekev
    k = 2 * np.sqrt(2*np.log(2))
    theta_sigma = theta_fwhm /k
    sigma0 = wl /(2*np.pi*theta_sigma)
    return sigma0*k

In [ ]:
#calculate angular divergence using formula from CDR2011
def calculate_theta_fwhm_cdr(ekev,qnC):
    theta_fwhm = (17.2 - 6.4 * np.sqrt(qnC))*1e-6/ekev**0.85
    return theta_fwhm

In [ ]:
#define optical path difference (OPD) from mirror profile, i.e.
#fill the struct opTrErMirr
#input: 
#    mdatafile: an ascii file with mirror profile data
#    ncol:      number of columns in the file
#    delim:     delimiter between numbers in an row, can be space (' '), tab '\t', etc
#    Orient:    mirror orientation, 'x' (horizontal) or 'y' (vertical)
#    theta:     incidence angle
#    scale:     scaling factor for the mirror profile
def defineOPD(opTrErMirr, mdatafile, ncol, delim, Orient, theta, scale):
    heightProfData = np.loadtxt(mdatafile).T
    AuxTransmAddSurfHeightProfileScaled(opTrErMirr, heightProfData, Orient, theta, scale)
    plt.figure()
    plot_1d(heightProfData,'profile from ' + mdatafile,'x (m)', 'h (m)') #@todo add the func def in on top of example

### Defining initial wavefront and writing electric field data to h5-file

In [ ]:
# #**********************Input Wavefront Structure and Parameters
print('*****defining initial wavefront and writing electric field data to h5-file...')
strInputDataFolder = 'data_common'  # input data sub-folder name
strOutputDataFolder = 'Tutorial_intro'  # output data sub-folder name

#init Gauusian beam parameters
d2m1_sase1 = 246.5
d2m1_sase2 = 290.0
d2m1_sase3 = 281.0
qnC = 0.1                    # e-bunch charge, [nC]
ekev_sase1 = 8.0
thetaOM_sase1 = 2.5e-3       # @check!
ekev_sase3 = 3.0
thetaOM_sase3 = 9.e-3

ekev = ekev_sase1
thetaOM = thetaOM_sase1
d2m1 = d2m1_sase1
#ekev = ekev_sase3
#thetaOM = thetaOM_sase3
#d2m1 = d2m1_sase3
z1 = d2m1
theta_fwhm = calculate_theta_fwhm_cdr(ekev,qnC)
k = 2*np.sqrt(2*np.log(2))
sigX = 12.4e-10*k/(ekev*4*np.pi*theta_fwhm) 
print('waist_fwhm [um], theta_fwhms [urad]:', sigX*k*1e6, theta_fwhm*1e6)
#define limits
range_xy = theta_fwhm/k*z1*7. # sigma*7 beam size
npoints=180

#define unique filename for storing results
ip = np.floor(ekev)
frac = np.floor((ekev - ip)*1e3)
fname0 = 'g' + str(int(ip))+'_'+str(int(frac))+'kev'
print('save hdf5: '+fname0+'.h5')
ifname = os.path.join(strOutputDataFolder,fname0+'.h5')

#build SRW gauusian wavefront
wfr0=build_gauss_wavefront_xy(nx=npoints,ny=npoints,ekev=ekev,
                              xMin=-range_xy/2, xMax=range_xy/2,
                              yMin=-range_xy/2, yMax=range_xy/2,
                              sigX=sigX, sigY=sigX, d2waist=z1)    
    


#init WPG Wavefront helper class
mwf = Wavefront(wfr0)

#store wavefront to HDF5 file 
mwf.store_hdf5(ifname)

#draw wavefront with common functions
plt.subplot(1,2,1)
plt.imshow(mwf.get_intensity(slice_number=0))
plt.subplot(1,2,2)
plt.imshow(mwf.get_phase(slice_number=0,polarization='horizontal'))
plt.show()

#draw wavefront with cuts
plot_wfront(mwf, title_fig='at '+str(z1)+' m',
            isHlog=False, isVlog=False,
            i_x_min=1e-5, i_y_min=1e-5, orient='x', onePlot=True,)

plt.set_cmap('bone') #set color map, 'bone', 'hot', 'jet', etc
fwhm_x = calculate_fwhm_x(mwf)
print('FWHMx [mm], theta_fwhm [urad]:',fwhm_x*1e3,fwhm_x/z1*1e6)

### Defining optical beamline(s) 

In [ ]:
print('*****Defining optical beamline(s) ...')
d2exp_sase1 = 904.0
d2exp_sase3 = 418.0

d2exp = d2exp_sase1
z2 = d2exp - d2m1
DriftM1_Exp = SRWLOptD(z2) #Drift from first offset mirror (M1) to exp hall
horApM1 = 0.8*thetaOM
opApM1 = SRWLOptA('r', 'a', horApM1, range_xy)  # clear aperture of the Offset Mirror(s)

#Wavefront Propagation Parameters:
#[0]:  Auto-Resize (1) or not (0) Before propagation
#[1]:  Auto-Resize (1) or not (0) After propagation
#[2]:  Relative Precision for propagation with Auto-Resizing (1. is nominal)
#[3]:  Allow (1) or not (0) for semi-analytical treatment of quadratic phase terms at propagation
#[4]:  Do any Resizing on Fourier side, using FFT, (1) or not (0)
#[5]:  Horizontal Range modification factor at Resizing (1. means no modification)
#[6]:  Horizontal Resolution modification factor at Resizing
#[7]:  Vertical Range modification factor at Resizing
#[8]:  Vertical Resolution modification factor at Resizing
#[9]:  Type of wavefront Shift before Resizing (not yet implemented)
#[10]: New Horizontal wavefront Center position after Shift (not yet implemented)
#[11]: New Vertical wavefront Center position after Shift (not yet implemented)
#                 [ 0] [1] [2]  [3] [4] [5]  [6]  [7]  [8]  [9] [10] [11] 
ppM1 =            [ 0,  0, 1.0,  0,  0, 1.0, 1.0, 1.0, 1.0,  0,  0,   0]
ppTrErM1 =        [ 0,  0, 1.0,  0,  0, 1.0, 1.0, 1.0, 1.0,  0,  0,   0]
ppDriftM1_Exp =   [ 0,  0, 1.0,  1,  0, 2.4, 1.8, 2.4, 1.8,  0,  0,   0]
ppLens =          [ 0,  0, 1.0,  0,  0, 1.0, 1.0, 1.0, 1.0,  0,  0,   0]
ppDrift_Foc  =    [ 0,  0, 1.0,  1,  0, 1.0, 1.5, 1.0, 1.5,  0,  0,   0]
ppFin  =          [ 0,  0, 1.0,  0,  0, 0.05,5.0, 0.05,5.0,  0,  0,   0]

optBL0 = SRWLOptC([opApM1,  DriftM1_Exp], 
                    [ppM1,ppDriftM1_Exp]) 

scale = 5     #5 mirror profile scaling factor 
print('*****HOM1 data for BL1 beamline ')
opTrErM1 = SRWLOptT(1500, 100, horApM1, range_xy)
defineOPD(opTrErM1, os.path.join(strInputDataFolder,'mirror1.dat'), 2, '\t', 'x',  thetaOM, scale)
opdTmp=np.array(opTrErM1.arTr)[1::2].reshape(opTrErM1.mesh.ny,opTrErM1.mesh.nx)
figure()
plot_2d(opdTmp, opTrErM1.mesh.xStart*1e3,opTrErM1.mesh.xFin*1e3,opTrErM1.mesh.yStart*1e3,opTrErM1.mesh.yFin*1e3,
        'OPD [m]', 'x (mm)', 'y (mm)')  

optBL1 = SRWLOptC([opApM1,opTrErM1,  DriftM1_Exp], 
                    [ppM1,ppTrErM1,ppDriftM1_Exp]) 

z3 = 30. #distance to focal plane
f_x = 1./(1./(z1+z2)+1./z3)
opLens = SRWLOptL(f_x,f_x,0,0)    
Drift_Foc = SRWLOptD(z3)
optBL2 = SRWLOptC([opApM1,opTrErM1,  DriftM1_Exp,opLens,  Drift_Foc], 
                    [ppM1,ppTrErM1,ppDriftM1_Exp,ppLens,ppDrift_Foc,ppFin]) 

optBL20= SRWLOptC([opApM1,  DriftM1_Exp,opLens,  Drift_Foc], 
                    [ppM1,ppDriftM1_Exp,ppLens,ppDrift_Foc,ppFin]) 


In [ ]:
print_beamline(optBL1)

### Propagating through BL0 beamline. Ideal mirror: HOM as an aperture

In [ ]:
print('*****Ideal mirror: HOM as an aperture')
bPlotted = False
isHlog = False
isVlog = False
bSaved = True
optBL = optBL0
strBL = 'bl0'
pos_title = 'at exp hall wall'
print('*****setting-up optical elements, beamline:', strBL)
print_beamline(optBL)
startTime = time.time()

print('*****reading wavefront from h5 file...')
w2 = Wavefront()
w2.load_hdf5(ifname)
wfr = w2._srwl_wf

print('*****propagating wavefront (with resizing)...')
srwl.PropagElecField(wfr, optBL)
mwf = Wavefront(wfr)
print('[nx, ny, xmin, xmax, ymin, ymax]', get_mesh(mwf))
if bSaved:
    print('save hdf5:', fname0+'_'+strBL+'.h5')
    mwf.store_hdf5(os.path.join(strOutputDataFolder, fname0+'_'+strBL+'.h5'))
print('done')
print('propagation lasted:', round((time.time() - startTime) / 6.) / 10., 'min')

In [ ]:
print('*****Ideal mirror: HOM as an aperture')
plot_wfront(mwf, 'at '+str(d2exp)+' m',False, False, 1e-5,1e-5,'x', True)
plt.set_cmap('bone') #set color map, 'bone', 'hot', 'jet', etc
plt.axis('tight')    
print('FWHMx [mm], theta_fwhm [urad]:',calculate_fwhm_x(mwf)*1e3,calculate_fwhm_x(mwf)/(z1+z2)*1e6)
print('FWHMy [mm], theta_fwhm [urad]:',calculate_fwhm_y(mwf)*1e3,calculate_fwhm_y(mwf)/(z1+z2)*1e6)

### Propagating through BL1 beamline. Imperfect mirror, unfocused beam

In [ ]:
print('*****Imperfect mirror, unfocused beam')
bPlotted = False
isHlog = True
isVlog = False
bSaved = False
optBL = optBL1
strBL = 'bl1'
pos_title = 'at exp hall wall'
print('*****setting-up optical elements, beamline:', strBL)
print_beamline(optBL)
startTime = time.time()
print('*****reading wavefront from h5 file...')
w2 = Wavefront()
w2.load_hdf5(ifname)
wfr = w2._srwl_wf
print('*****propagating wavefront (with resizing)...')
srwl.PropagElecField(wfr, optBL)
mwf = Wavefront(wfr)
print('[nx, ny, xmin, xmax, ymin, ymax]', get_mesh(mwf))
if bSaved:
    print('save hdf5:', fname0+'_'+strBL+'.h5')
    mwf.store_hdf5(os.path.join(strOutputDataFolder,fname0+'_'+strBL+'.h5'))
print('done')
print('propagation lasted:', round((time.time() - startTime) / 6.) / 10., 'min')

In [ ]:
print ('*****Imperfect mirror, unfocused beam')
plot_wfront(mwf, 'at '+str(d2exp)+' m',False, False, 1e-5,1e-5,'x', True)
plt.set_cmap('bone') #set color map, 'bone', 'hot', etc
plt.axis('tight')    
print('FWHMx [mm], theta_fwhm [urad]:',calculate_fwhm_x(mwf)*1e3,calculate_fwhm_x(mwf)/(z1+z2)*1e6)
print('FWHMy [mm], theta_fwhm [urad]:',calculate_fwhm_y(mwf)*1e3,calculate_fwhm_y(mwf)/(z1+z2)*1e6)

### Propagating through BL2 beamline. Focused beam: HOM aperture effect

In [ ]:
print ('*****Focused beam: Focused beam: HOM aperture effect')
bPlotted = False
isHlog = True
isVlog = False
bSaved = False
optBL = optBL20
strBL = 'bl20'
pos_title = 'at sample'
print('*****setting-up optical elements, beamline:', strBL)
print_beamline(optBL)
startTime = time.time()
print('*****reading wavefront from h5 file...')
w2 = Wavefront()
w2.load_hdf5(ifname)
wfr = w2._srwl_wf
print('*****propagating wavefront (with resizing)...')
srwl.PropagElecField(wfr, optBL)
mwf = Wavefront(wfr)
print('[nx, ny, xmin, xmax, ymin, ymax]', get_mesh(mwf))
if bSaved:
    print('save hdf5:', fname0+'_'+strBL+'.h5')
    mwf.store_hdf5(os.path.join(strOutputDataFolder,fname0+'_'+strBL+'.h5'))
print('done')
print('propagation lasted:', round((time.time() - startTime) / 6.) / 10., 'min')

In [ ]:
print ('*****Focused beam: HOM aperture effect')
plot_wfront(mwf, 'at '+str(d2exp)+' m',False, False, 1e-5,1e-5,'x', True)
plt.set_cmap('bone') #set color map, 'bone', 'hot', etc
plt.axis('tight')    
print('FWHMx [mm], FWHMy [mm]:',calculate_fwhm_x(mwf)*1e3,calculate_fwhm_y(mwf)*1e3)

### Propagating through BL3 beamline. Focused beam: Imperfect mirror

In [ ]:
print ('*****Focused beam: Imperfect mirror')
bPlotted = False
isHlog = True
isVlog = False
bSaved = False
optBL = optBL2
strBL = 'bl2'
pos_title = 'at sample position'
print('*****setting-up optical elements, beamline:', strBL)
print_beamline(optBL)
startTime = time.time()
print('*****reading wavefront from h5 file...')
w2 = Wavefront()
w2.load_hdf5(ifname)
wfr = w2._srwl_wf
print('*****propagating wavefront (with resizing)...')
srwl.PropagElecField(wfr, optBL)
mwf = Wavefront(wfr)
print('[nx, ny, xmin, xmax, ymin, ymax]', get_mesh(mwf))
if bSaved:
    print('save hdf5:', fname0+'_'+strBL+'.h5')
    mwf.store_hdf5(os.path.join(strOutputDataFolder,fname0+'_'+strBL+'.h5'))
print('done')
print('propagation lasted:', round((time.time() - startTime) / 6.) / 10., 'min')

In [ ]:
print('*****Focused beam: Imperfect mirror')
plot_wfront(mwf, 'at '+str(d2exp)+' m',False, False, 1e-5,1e-5,'x', True)
plt.set_cmap('bone') #set color map, 'bone', 'hot', etc
plt.axis('tight')    
print('FWHMx [mm], FWHMy [mm]:',calculate_fwhm_x(mwf)*1e3,calculate_fwhm_y(mwf)*1e3)
#int_x=mwf.get_intensity(slice=0); np.savetxt('bl2.txt',int_x[int_x.shape[1]/2,:])

### Propagating through BL4 beamline. Focused beam, out of focus

In [ ]:
print('*****Focused beam, out of focus')
dz = 15.e-3
n = 5
startTime = time.time()
for idx in range(-n,n):
    Drift = SRWLOptD(z3+dz*idx/n)
    optBL = SRWLOptC([opApM1,opTrErM1,  DriftM1_Exp,opLens,  Drift], 
                       [ppM1,ppTrErM1,ppDriftM1_Exp,ppLens,ppDrift_Foc,ppFin]) 
    #print_beamline(optBL4)
    strBL = 'bl'+'_'+str(idx+n)
    print('*****reading wavefront from h5 file...')
    w2 = Wavefront()
    w2.load_hdf5(ifname)
    wfr = w2._srwl_wf
    print('*****propagating wavefront (with resizing)...')
    srwl.PropagElecField(wfr, optBL)
    mwf = Wavefront(wfr)
    print('save hdf5:', fname0+'_'+strBL+'.h5')
    mwf.store_hdf5(os.path.join(strOutputDataFolder,fname0+'_'+strBL+'.h5'))


In [ ]:
print('*****Focused beam, out of focus, last slice')
plot_wfront(mwf, 'at '+str(d2exp)+' m',False, False, 1e-5,1e-5,'x', True)
plt.set_cmap('bone') #set color map, 'bone', 'hot', etc
plt.axis('tight')    
print('FWHMx [um], FWHMy [um]:',calculate_fwhm_x(mwf)*1e6,calculate_fwhm_y(mwf)*1e6)
#int_x=mwf.get_intensity(slice=0); np.savetxt('bl2.txt',int_x[int_x.shape[1]/2,:])